# Compute abundance change for each allele

This script takes in the single-cell FACS data as input, applies various filtering criteria, and then produces a z-score for the abundance of each allele relative to the reference on the same plate.

In [1]:
import pandas as pd
import numpy as np
from random import choice
import matplotlib.pyplot as plt
import polars as pl
import process_dualipa as proc

In [2]:
dualipa_inputs = "../1_inputs"
dualipa_outputs = "../3_outputs"
meta_outputs = "../../../1_allele_collection/3_outputs"

n_cell_threshold = 800
wt_gfp_threshold = 100

In [3]:
# Read in single-cell measurements
pDEST_DUAL_df = pd.read_parquet(f"{dualipa_outputs}/facs_single_cell.parquet")

# Filter to only keep wells with more than 800 cells
pDEST_DUAL_df = pDEST_DUAL_df[pDEST_DUAL_df['n_cells'] > n_cell_threshold]

In [4]:
# get the mean and median measurements per well
keep_cols = ['symbol', 'node_type', 'nt_change','aa_change', 'pla', 'well', 'coordinates', 'n_cells', 'orf_id', 'mut_id', 'valid_well']
pDEST_DUAL_avg_df = pDEST_DUAL_df[keep_cols + ['avg_gfp', 'avg_mcherry','avg_GFP_mCherry_ratio']].drop_duplicates()
pDEST_DUAL_median_df = pDEST_DUAL_df[keep_cols + ['median_gfp', 'median_mcherry','median_GFP_mCherry_ratio']].drop_duplicates()

## 1. Compute the mean and mediam scores

__Functions for computing is in ```process_dualipa.py```__

Maxime's notes:

    Instead of using Georges' approach to compute the assay's variability, which uses a step with random pairings, 
    Luke suggested to compute a STD from the Log2FC of all individual WT measurements, relative to the mean of the WT of each gene.
    This is a more robust approach, as it does not rely on random pairings.
    This function computes the STD of the Log2FC of all individual WT measurements, relative to the mean of the WT of each gene.
    Returns a tuple with the mean and the STD of the Log2FC of all individual WT measurements, relative to the mean of the WT of each gene.

In [5]:
# _d objects: mean or median GFP_mCherry ratio of each unique (wt orf, plate) combination
# _l objects: list of WT:WT abundances (GFP:mCherry ratios) to estimate assay variability
wt_avg_d, wt_ratio_l = proc.get_wt_variability_d(pDEST_DUAL_avg_df, pDEST_DUAL_df)
wt_median_d, wt_ratio_l_median = proc.get_wt_variability_d_median(pDEST_DUAL_median_df, pDEST_DUAL_df)

In [6]:
wt_std, wt_mean = proc.wt_log2fc_variability(pDEST_DUAL_avg_df, pDEST_DUAL_df)

In [7]:
# Must be two WT replicates on same plate to compute the ratio, otherwise we return a NaN
pDEST_DUAL_avg_allele_df = proc.get_pDEST_DUAL_avg_allele_df(pDEST_DUAL_avg_df, wt_avg_d, wt_ratio_l, wt_std, wt_mean)
pDEST_DUAL_median_allele_df = proc.get_pDEST_DUAL_median_allele_df(pDEST_DUAL_median_df, wt_median_d, wt_ratio_l_median)

In [9]:
mean_df = pl.DataFrame(pDEST_DUAL_avg_allele_df)
median_df = pl.DataFrame(pDEST_DUAL_median_allele_df)

# append metadata and write out files
metadata = pl.read_csv(f"{meta_outputs}/varchamp_alleles_with_pillar_annotations.tsv", separator="\t", infer_schema_length=1000000).with_columns(
    pl.col("orf_id_wt").cast(pl.Float64).alias("orf_id"),
    pl.col("mutation_id_old").cast(pl.Float64).alias("mut_id")
)

In [10]:
mean_df = mean_df.join(
    metadata.select(pl.col(["orf_id","mut_id"]+[c for c in metadata.columns if c not in mean_df.columns])), on=["orf_id", "mut_id"], how="left")
median_df = median_df.join(
    metadata.select(pl.col(["orf_id","mut_id"]+[c for c in metadata.columns if c not in median_df.columns])), on=["orf_id", "mut_id"], how="left")

In [12]:
median_df

symbol,node_type,nt_change,aa_change,pla,well,coordinates,n_cells,orf_id,mut_id,valid_well,median_gfp,median_mcherry,median_GFP_mCherry_ratio,wt_GFP_mCherry_ratio_median,allele_wt_ratio_median,zscore_median,zcat_median,zscore_log2,ensembl_gene_id,orf_id_wt,mutation_id_old,ccsb_mutation_id,ccsb_allele_id,spdi,collection,entry_plate_orig,entry_well_orig,entry_plate_conso,entry_well_conso,entry_seq_pool,db_plate,db_well,n2h_plate,n2h_well,dualip_plate,dualip_well,…,Calcualted_post_prob_PMID:31131967,calculated_classification_PMID:31131967,SGR LR_PMID:34273903,Personal and Family History LR (Combined)_PMID:34273903,Breast Tumor Pathology LR_PMID:34273903,Population Allele Frequency LR_PMID:34273903,BS2 LR_PMID:34273903,Calculated_combined_LR_PMID:34273903,Naive_prior_prob_PMID:34273903,Calculated_post_prob_PMID:34273903,calculated_classification_PMID:34273903,Reference Group*_presumed_PMID:34273903,Component_presumed_PMID:34273903,Rationale_presumed_PMID:34273903,ClinVar Variation Id_ClinGen_repo,Allele Registry Id_ClinGen_repo,Disease_ClinGen_repo,Mondo Id_ClinGen_repo,Mode of Inheritance_ClinGen_repo,Assertion_ClinGen_repo,Applied Evidence Codes (Met)_ClinGen_repo,Applied Evidence Codes (Not Met)_ClinGen_repo,Summary of interpretation_ClinGen_repo,PubMed Articles_ClinGen_repo,Expert Panel_ClinGen_repo,Guideline_ClinGen_repo,Approval Date_ClinGen_repo,Published Date_ClinGen_repo,Retracted_ClinGen_repo,Evidence Repo Link_ClinGen_repo,Uuid_ClinGen_repo,Updated_Classification_ClinGen_repo,Updated_Evidence Codes_ClinGen_repo,gene_variant_pp,aa_change_pp,pillar_tested,spdi_trimmed_pp
str,str,str,str,i64,str,str,i64,f64,f64,bool,f64,f64,f64,f64,f64,f64,i64,f64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,bool,str
"""IL36RN""","""allele""","""368C>G""","""Thr123Arg""",3,"""C09""","""3_C09""",9149,8151.0,202847.0,true,32.760002,3332.0,0.008973,null,null,null,0,null,"""ENSG00000136695""",8151,202847,"""CCSBVarC006168""","""ALE00006168""","""NC_000002.12:113062577:C:G""","""CEGS2""","""CegsMutGDEh1036""","""E07""","""GDEhDisVCh_40084""","""B05""","""2""","""CegsMutGDDh1036""","""E07""","""VUSN2Hmut_GDN2h_07""","""E05""","""VUSMutpDEST2_07""","""F09""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""MLH1""","""allele""","""116G>T""","""Cys39Phe""",18,"""H02""","""18_H02""",8459,2713.0,473.0,true,46.5,5552.399902,0.008741,null,null,null,0,null,"""ENSG00000076242""",2713,473,"""CCSBVarC007377""","""ALE00007365""","""NC_000003.12:36993663:G:T""","""Edgotyping3""","""VUSMutGDEh011""","""F08""","""GDEhDisVCh_40019""","""A06""","""1""","""VUSMutGDDh011""","""F08""","""VUSN2Hmut_GDN2h_04""","""A01""","""VUSMutpDEST2_02""","""F03""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KCNJ2""","""allele""","""821A>T""","""Asp274Val""",11,"""H06""","""11_H06""",4976,1.00016069e8,2638.0,true,897.920013,2918.650024,0.253414,0.841771,0.301048,-1.871967,-1,-3.706326,"""ENSG00000123700""",100016069,2638,"""CCSBVarC008340""","""ALE00008326""","""NC_000017.11:70175860:A:T""","""Edgotyping3""","""VUSMutGDEh010""","""D06""","""GDEhDisVCh_40005""","""C10""","""1""","""VUSMutGDDh010""","""D06""","""VUSN2Hmut_GDN2h_03""","""F09""","""VUSMutpDEST2_16""","""D04""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""PLP1""","""allele""","""128C>T""","""Thr43Ile""",9,"""B11""","""9_B11""",9258,4062.0,201422.0,true,126.040001,5390.349854,0.024643,null,null,null,0,null,"""ENSG00000123560""",

In [13]:
mean_df.write_csv(f'{dualipa_outputs}/DUALIPA_mean_zscore.csv')
median_df.write_csv(f'{dualipa_outputs}/DUALIPA_median_zscore.csv')